# DATA 691 - Summer 2025 - Assignment 2

### 1. Fashion MNIST

Download the [Fashion MNIST dataset](https://www.kaggle.com/datasets/zalando-research/fashionmnist) from Kaggle.

Load the 60,000 training examples (`fashion-mnist_train.csv`) and split it into equal size ***training*** and ***validation*** subsets.

Your goal is to fit the best model you can to the training subset of size 30,000 constructed above, where best means highest accuracy on the validation set of size 30,000 constructed above.

Your models should be pipelines with constituents chosen among the following:

- `sklearn.preprocessing.StandardScaler`

- `sklearn.preprocessing.Normalizer`

- `sklearn.decomposition.PCA`

- `sklearn.linear_model.LogisticRegression`

- `sklearn.svm.SVC`

You don't have to use all of them!

Restrict your parameter tuning to the the regularization parameter `C` of `LogisticRegression`/`SVC` and the `n_components` parameter of `PCA`.

I suggest starting your analysis with a small subset of the training data, growing it as you identify good combinations of pipeline constituents and parameters.

Once you have settled on an optimal model, evaluate it on the test set (`fashion-mnist_test.csv`) and report the corresponding accuracy.

In [ ]:
# Here are the labels.

labels = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

### 2. SMS spam filtering

In this activity, we'll try to flag spam SMS messages based on the text of the message.

The dataset comes from the UCI Machine Learning Repository ([link](https://archive.ics.uci.edu/dataset/228/sms+spam+collection)).

I preprocessed it a bit. Load it from `data/sms_spam.csv`.

#### Manual feature extraction

If you examine some of the messages in the dataset, you'll notice some patterns that you might exploit for classifying spam messages. 

Add the following features to the dataframe:

- `length`, the length of a message, in characters,

- `num_caps`, the number of capital letters in a message,

- `proportion_caps` the proportion of capital letters in a message,

- `num_digits`, the number of digits in a message,

- `proportion_digits` the proportion of digits in a message,

- binary features `contains_<char>` indicating whether each of the following characters occurs in a message:
`@`, `#`, `$`, `*`, `/`, `:`, `-`, `+`, `£`, `(`, `)`, `[`, `]`, `;`, `<`, `>`, `?`

Compute cross-validated accuracy, $F_1$, precision, and recall metrics for `LogisticRegression`, `SGDClassifier`, and `LinearSVC` models fit to the the data using these features that you extracted.
If you do this using `cross_val_score`, you'll need to loop over the metrics yourself. If you use [`cross_validate`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html) instead (try it!) you can pass in a list of metrics.

How do the models compare across the various metrics?

#### Feature extraction with `CountVectorizer`

Compute cross-validated accuracy, $F_1$, precision, and recall metrics for `LogisticRegression`, `SGDClassifier`, `LinearSVC`, and `MultinomialNB` models fit to the data, extracting features using `CountVectorizer`.
How do the models compare across the various metrics?

Can you improve performance by tuning parameters of the vectorizer or the classifier? Does swapping out `CountVectorizer` with `TfidfVectorizer` improve any metrics?
